# Edges round regression (Ridge)

Predict draft round 1–8 (8 = undrafted) using combine+PFF features, KNN imputation, Ridge regression.
- Train: 2015–2023 (edge_training.csv).
- Test: edges_drafted_2024.csv, edges_drafted_2025.csv (drafted only; actual rounds 1–7).

In [19]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

FEATURES_WITH_COLLEGE = [
    'Broad Jump', 'Vertical', '40yd', 'Height', 'Weight',
    'speed_score', 'explosive_score', 'RAS', 'arm_length_inches',
    'true_pass_set_pass_rush_win_rate', 'pass_rush_win_rate', 'snap_counts_pass_rush', 'stop_percent', 'p4_conference'
]
CONTAINS_WITH_COLLEGE = [
    'contains_broad_jump', 'contains_vertical', 'contains_40yd', 'contains_height', 'contains_weight',
    'contains_speed_score', 'contains_explosive_score', 'contains_ras', 'contains_arm_length_inches',
    'contains_true_pass_set_pass_rush_win_rate', 'contains_pass_rush_win_rate', 'contains_snap_counts_pass_rush', 'contains_stop_percent',
    'contains_p4_conference'
]
FEATURES_WITH_COLLEGE_ALL = FEATURES_WITH_COLLEGE + CONTAINS_WITH_COLLEGE

In [20]:
# Load edge training data (2015–2023)
df = pd.read_csv('../data/processed/edge_training.csv')
df = df[df['Year'].between(2015, 2023)].copy()
# RAS already included in edge_training.csv
print('Train (2015–2023 edges):', len(df))

Train (2015–2023 edges): 240


In [21]:
# Print RAS score availability
ras_count = df['RAS'].notna().sum()
total_count = len(df)
ras_pct = (ras_count / total_count * 100) if total_count > 0 else 0
print(f"Players with RAS score: {ras_count} out of {total_count} ({ras_pct:.1f}%)")

# Print PFF metrics availability
pff_true_pass_set_count = df['true_pass_set_pass_rush_win_rate'].notna().sum()
pff_pass_rush_win_count = df['pass_rush_win_rate'].notna().sum()
pff_snap_counts_count = df['snap_counts_pass_rush'].notna().sum()
pff_true_pass_set_pct = (pff_true_pass_set_count / total_count * 100) if total_count > 0 else 0
pff_pass_rush_win_pct = (pff_pass_rush_win_count / total_count * 100) if total_count > 0 else 0
pff_snap_counts_pct = (pff_snap_counts_count / total_count * 100) if total_count > 0 else 0
print(f"Players with True Pass Set Win Rate: {pff_true_pass_set_count} out of {total_count} ({pff_true_pass_set_pct:.1f}%)")
print(f"Players with Pass Rush Win Rate: {pff_pass_rush_win_count} out of {total_count} ({pff_pass_rush_win_pct:.1f}%)")
print(f"Players with Pass Rush Snap Counts: {pff_snap_counts_count} out of {total_count} ({pff_snap_counts_pct:.1f}%)")
pff_stop_count = df['stop_percent'].notna().sum()
pff_stop_pct = (pff_stop_count / total_count * 100) if total_count > 0 else 0
print(f"Players with Run Defense Stop %: {pff_stop_count} out of {total_count} ({pff_stop_pct:.1f}%)")
arm_count = df['arm_length_inches'].notna().sum() if 'arm_length_inches' in df.columns else 0
arm_pct = (arm_count / total_count * 100) if total_count > 0 else 0
print(f"Players with arm length: {arm_count} out of {total_count} ({arm_pct:.1f}%)")

Players with RAS score: 132 out of 240 (55.0%)
Players with True Pass Set Win Rate: 188 out of 240 (78.3%)
Players with Pass Rush Win Rate: 188 out of 240 (78.3%)
Players with Pass Rush Snap Counts: 188 out of 240 (78.3%)
Players with Run Defense Stop %: 159 out of 240 (66.2%)
Players with arm length: 228 out of 240 (95.0%)


In [22]:
# Height to inches
def height_inches(h):
    if pd.isna(h): return np.nan
    if isinstance(h, (int, float)) and not (isinstance(h, float) and np.isnan(h)):
        return float(h)
    s = str(h).strip()
    if '-' in s:
        parts = s.split('-')
        return int(parts[0]) * 12 + int(parts[1])
    return np.nan
df['Height'] = df['Height'].apply(height_inches)

# Speed score
df['speed_score'] = np.where(
    df['40yd'].notna() & (df['40yd'] > 0),
    df['Weight'] * 200 / (df['40yd'] ** 4),
    np.nan
)

# Explosive score (z-scores from this pool)
mean_v = df['Vertical'].mean()
std_v = df['Vertical'].std()
mean_b = df['Broad Jump'].mean()
std_b = df['Broad Jump'].std()
if std_v == 0 or np.isnan(std_v): std_v = 1.0
if std_b == 0 or np.isnan(std_b): std_b = 1.0
df['explosive_score'] = (df['Vertical'] - mean_v).fillna(0) / std_v + (df['Broad Jump'] - mean_b).fillna(0) / std_b

# P4 conference (from defensive_stats)
P4_WITH_PAC12 = {'SEC', 'Big Ten', 'Big 12', 'ACC', 'Pac-12'}
P4_NO_PAC12 = {'SEC', 'Big Ten', 'Big 12', 'ACC'}
school_alias = {
    'Ole Miss': 'Mississippi', 'Miami (FL)': 'Miami', 'Southern California': 'USC',
    'Central Florida': 'UCF', 'Brigham Young': 'BYU', 'Ohio St.': 'Ohio State',
    'Florida St.': 'Florida State', 'Kansas St.': 'Kansas State', 'Iowa St.': 'Iowa State',
    'Oklahoma St.': 'Oklahoma State', 'Penn St.': 'Penn State', 'San Diego St.': 'San Diego State',
}
_stats = pd.read_csv('../data/processed/defensive_stats_2016_to_2025.csv')
P4_SCHOOLS = set(_stats[_stats['Conference'].isin(P4_WITH_PAC12)]['Team'].unique())
P4_SCHOOLS_NO_PAC12 = set(_stats[_stats['Conference'].isin(P4_NO_PAC12)]['Team'].unique())

def is_p4(row):
    s = row.get('School')
    if pd.isna(s) or s == '': return 0
    sn = school_alias.get(s, s)
    year = row.get('Year', 2023)
    schools = P4_SCHOOLS if year <= 2023 else P4_SCHOOLS_NO_PAC12
    return 1 if sn in schools else 0
df['p4_conference'] = df.apply(is_p4, axis=1)

# Contains flags
df['contains_broad_jump'] = df['Broad Jump'].notna().astype(int)
df['contains_vertical'] = df['Vertical'].notna().astype(int)
df['contains_40yd'] = df['40yd'].notna().astype(int)
df['contains_height'] = df['Height'].notna().astype(int)
df['contains_weight'] = df['Weight'].notna().astype(int)
df['contains_speed_score'] = df['speed_score'].notna().astype(int)
df['contains_explosive_score'] = 1  # we always have it (filled 0)
df['contains_true_pass_set_pass_rush_win_rate'] = df['true_pass_set_pass_rush_win_rate'].notna().astype(int)
df['contains_pass_rush_win_rate'] = df['pass_rush_win_rate'].notna().astype(int)
df['contains_snap_counts_pass_rush'] = df['snap_counts_pass_rush'].notna().astype(int)
df['contains_stop_percent'] = df['stop_percent'].notna().astype(int)
df['contains_ras'] = df['RAS'].notna().astype(int)
if 'arm_length_inches' not in df.columns:
    df['arm_length_inches'] = np.nan
df['contains_arm_length_inches'] = df['arm_length_inches'].notna().astype(int)
df['contains_p4_conference'] = df['School'].notna().astype(int)

In [23]:
# Target: round 1–7 if drafted, 8 if undrafted
y = np.where(
    df['Drafted'].astype(bool),
    np.clip(df['Round'].fillna(1).astype(int), 1, 7),
    8
)
X_raw = df[FEATURES_WITH_COLLEGE_ALL].copy()

# KNN imputation + scale
imputer = KNNImputer(n_neighbors=10)
X = imputer.fit_transform(X_raw)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Ridge regression
ridge = Ridge(alpha=1.0, random_state=42)
ridge.fit(X_scaled, y)

y_pred_train = np.clip(ridge.predict(X_scaled), 1, 8)
print('Train MAE (round 1–8):', round(mean_absolute_error(y, y_pred_train), 4))
print('Train samples:', len(y))

Train MAE (round 1–8): 1.6324
Train samples: 240


In [24]:
def prepare_edges_df(csv_path, year):
    """Load edges CSV and build same features (Height inches, speed_score, explosive_score, p4, contains_*).
    PFF data should already be in the CSV file.
    """
    edf = pd.read_csv(csv_path)
    edf['Year'] = year
    if edf['Height'].dtype == object or (edf['Height'].astype(str).str.contains('-', na=False).any()):
        edf['Height'] = edf['Height'].apply(height_inches)
    else:
        edf['Height'] = pd.to_numeric(edf['Height'], errors='coerce')
    edf['speed_score'] = np.where(
        edf['40yd'].notna() & (edf['40yd'] > 0),
        edf['Weight'] * 200 / (edf['40yd'] ** 4),
        np.nan
    )
    edf['explosive_score'] = (edf['Vertical'] - mean_v).fillna(0) / std_v + (edf['Broad Jump'] - mean_b).fillna(0) / std_b
    edf['p4_conference'] = edf.apply(is_p4, axis=1)
    # RAS already included in edge_testing.csv
    edf['contains_broad_jump'] = edf['Broad Jump'].notna().astype(int)
    edf['contains_vertical'] = edf['Vertical'].notna().astype(int)
    edf['contains_40yd'] = edf['40yd'].notna().astype(int)
    edf['contains_height'] = edf['Height'].notna().astype(int)
    edf['contains_weight'] = edf['Weight'].notna().astype(int)
    edf['contains_speed_score'] = edf['speed_score'].notna().astype(int)
    edf['contains_explosive_score'] = 1
    edf['contains_true_pass_set_pass_rush_win_rate'] = edf['true_pass_set_pass_rush_win_rate'].notna().astype(int)
    edf['contains_pass_rush_win_rate'] = edf['pass_rush_win_rate'].notna().astype(int)
    edf['contains_snap_counts_pass_rush'] = edf['snap_counts_pass_rush'].notna().astype(int)
    edf['contains_stop_percent'] = edf['stop_percent'].notna().astype(int) if 'stop_percent' in edf.columns else 0
    # Handle RAS column (should be in edge_testing.csv, but add if missing)
    if 'RAS' not in edf.columns:
        edf['RAS'] = np.nan
    edf['contains_ras'] = edf['RAS'].notna().astype(int)
    if 'arm_length_inches' not in edf.columns:
        edf['arm_length_inches'] = np.nan
    edf['contains_arm_length_inches'] = edf['arm_length_inches'].notna().astype(int)
    edf['contains_p4_conference'] = edf['School'].notna().astype(int)
    return edf

# 2024 and 2025 (drafted only; actual round 1–7) - load from edge_testing.csv which has PFF data
edge_testing = pd.read_csv('../data/processed/edge_testing.csv')
edges_2024 = edge_testing[edge_testing['Year'] == 2024].copy()
edges_2025 = edge_testing[edge_testing['Year'] == 2025].copy()
# Prepare features for 2024/2025 (they already have PFF data, just need feature engineering)
def prepare_test_edges(edf):
    if edf['Height'].dtype == object or (edf['Height'].astype(str).str.contains('-', na=False).any()):
        edf['Height'] = edf['Height'].apply(height_inches)
    else:
        edf['Height'] = pd.to_numeric(edf['Height'], errors='coerce')
    edf['speed_score'] = np.where(
        edf['40yd'].notna() & (edf['40yd'] > 0),
        edf['Weight'] * 200 / (edf['40yd'] ** 4),
        np.nan
    )
    edf['explosive_score'] = (edf['Vertical'] - mean_v).fillna(0) / std_v + (edf['Broad Jump'] - mean_b).fillna(0) / std_b
    edf['p4_conference'] = edf.apply(is_p4, axis=1)
    # RAS already included in edge_testing.csv
    # Add contains flags
    edf['contains_broad_jump'] = edf['Broad Jump'].notna().astype(int)
    edf['contains_vertical'] = edf['Vertical'].notna().astype(int)
    edf['contains_40yd'] = edf['40yd'].notna().astype(int)
    edf['contains_height'] = edf['Height'].notna().astype(int)
    edf['contains_weight'] = edf['Weight'].notna().astype(int)
    edf['contains_speed_score'] = edf['speed_score'].notna().astype(int)
    edf['contains_explosive_score'] = 1
    edf['contains_true_pass_set_pass_rush_win_rate'] = edf['true_pass_set_pass_rush_win_rate'].notna().astype(int)
    edf['contains_pass_rush_win_rate'] = edf['pass_rush_win_rate'].notna().astype(int)
    edf['contains_snap_counts_pass_rush'] = edf['snap_counts_pass_rush'].notna().astype(int)
    edf['contains_stop_percent'] = edf['stop_percent'].notna().astype(int) if 'stop_percent' in edf.columns else 0
    edf['contains_ras'] = edf['RAS'].notna().astype(int)
    if 'arm_length_inches' not in edf.columns:
        edf['arm_length_inches'] = np.nan
    edf['contains_arm_length_inches'] = edf['arm_length_inches'].notna().astype(int)
    edf['contains_p4_conference'] = edf['School'].notna().astype(int)
    return edf
edges_2024 = prepare_test_edges(edges_2024)
edges_2025 = prepare_test_edges(edges_2025)

X_24_raw = edges_2024[FEATURES_WITH_COLLEGE_ALL].copy()
X_25_raw = edges_2025[FEATURES_WITH_COLLEGE_ALL].copy()
X_24 = imputer.transform(X_24_raw)
X_25 = imputer.transform(X_25_raw)
X_24_scaled = scaler.transform(X_24)
X_25_scaled = scaler.transform(X_25)

pred_24 = np.clip(ridge.predict(X_24_scaled), 1, 8)
pred_25 = np.clip(ridge.predict(X_25_scaled), 1, 8)

actual_24 = edges_2024['Round'].astype(int).values
actual_25 = edges_2025['Round'].astype(int).values

def eval_metrics(actual, pred, label):
    mae = mean_absolute_error(actual, pred)
    rmse = np.sqrt(mean_squared_error(actual, pred))
    r2 = r2_score(actual, pred)
    exact = (np.round(pred) == actual).mean()
    within_1 = (np.abs(np.round(pred) - actual) <= 1).mean()
    print(f'{label} (n={len(actual)}): MAE={mae:.4f}, RMSE={rmse:.4f}, R²={r2:.4f}, Exact={exact:.2%}, Within-1={within_1:.2%}')

print('2024 edges:')
eval_metrics(actual_24, pred_24, '2024')
print('2025 edges:')
eval_metrics(actual_25, pred_25, '2025')

2024 edges:
2024 (n=18): MAE=1.0835, RMSE=1.2726, R²=0.6466, Exact=16.67%, Within-1=77.78%
2025 edges:
2025 (n=14): MAE=1.5303, RMSE=1.8632, R²=-3.1488, Exact=7.14%, Within-1=71.43%


In [25]:
# Dataframes: players with actual round, model prediction, tier label, and interpretation
def pred_round_to_tier(p):
    if p < 1.75: return ('Round 1 Tier', 'True 1st-round grade')
    if p < 2.75: return ('Round 2 Tier', 'Early Day 2')
    if p < 3.75: return ('Round 3 Tier', 'Late Day 2')
    if p < 4.75: return ('Round 4 Tier', 'Early Day 3')
    if p < 5.75: return ('Round 5 Tier', 'Mid Day 3')
    if p < 6.75: return ('Round 6 Tier', 'Late Day 3')
    return ('Round 7 / UDFA Tier', 'Fringe draftable')

edges_2024_display = edges_2024[['Round', 'Pick', 'Player', 'School', 'Year', 'RAS']].copy()
edges_2024_display['predicted_round'] = pred_24
edges_2024_display['tier_label'] = [pred_round_to_tier(x)[0] for x in pred_24]
edges_2024_display['interpretation'] = [pred_round_to_tier(x)[1] for x in pred_24]
edges_2024_display['Round'] = edges_2024_display['Round'].astype(int)
# Add contains flags
edges_2024_display['contains_ras'] = edges_2024['contains_ras'].values
edges_2024_display['contains_pff'] = (
    edges_2024['contains_true_pass_set_pass_rush_win_rate'] | 
    edges_2024['contains_pass_rush_win_rate'] | 
    edges_2024['contains_snap_counts_pass_rush'] | 
    edges_2024['contains_stop_percent']
).astype(int)

edges_2025_display = edges_2025[['Round', 'Pick', 'Player', 'School', 'Year', 'RAS']].copy()
edges_2025_display['predicted_round'] = pred_25
edges_2025_display['tier_label'] = [pred_round_to_tier(x)[0] for x in pred_25]
edges_2025_display['interpretation'] = [pred_round_to_tier(x)[1] for x in pred_25]
edges_2025_display['Round'] = edges_2025_display['Round'].astype(int)
# Add contains flags
edges_2025_display['contains_ras'] = edges_2025['contains_ras'].values
edges_2025_display['contains_pff'] = (
    edges_2025['contains_true_pass_set_pass_rush_win_rate'] | 
    edges_2025['contains_pass_rush_win_rate'] | 
    edges_2025['contains_snap_counts_pass_rush'] | 
    edges_2025['contains_stop_percent']
).astype(int)

print('2024 drafted edges')
display(edges_2024_display)
print('2025 drafted edges')
display(edges_2025_display)

2024 drafted edges


,Round,Pick,Player,School,Year,RAS,predicted_round,tier_label,interpretation,contains_ras,contains_pff
0,1,15,Laiatu Latu,UCLA,2024,9.37,1.803220,Round 2 Tier,Early Day 2,1,1
1,1,17,Dallas Turner,Alabama,2024,8.86,2.432544,Round 2 Tier,Early Day 2,1,1
2,1,19,Jared Verse,Florida State,2024,9.60,1.361540,Round 1 Tier,True 1st-round grade,1,1
3,1,21,Demeioun Robinson,Penn State,2024,9.72,3.456787,Round 3 Tier,Late Day 2,1,1
4,2,56,Marshawn Kneeland,Western Michigan,2024,9.08,4.300427,Round 4 Tier,Early Day 3,1,1
5,2,57,Chris Braswell,Alabama,2024,8.73,3.788644,Round 4 Tier,Early Day 3,1,1
6,3,74,Bralen Trice,Washington,2024,7.46,2.655636,Round 2 Tier,Early Day 2,1,1
7,3,76,Jonah Elliss,Utah,2024,8.59,3.864038,Round 4 Tier,Early Day 3,1,1
8,3,94,Jalyx Hunt,Houston Christian,2024,9.22,4.096354,Round 4 Tier,Early Day 3,1,0
9,5,138,Xavier Thomas,Clemson,2024,8.17,4.196811,Round 4 Tier,Early Day 3,1,1


2025 drafted edges


,Round,Pick,Player,School,Year,RAS,predicted_round,tier_label,interpretation,contains_ras,contains_pff
18,1,3,Abdul Carter,Penn State,2025,NaN,2.065812,Round 2 Tier,Early Day 2,0,1
19,1,11,Mykel Williams,Georgia,2025,NaN,4.445284,Round 4 Tier,Early Day 3,0,1
20,1,17,Shemar Stewart,Texas A&M,2025,10.00,2.190173,Round 2 Tier,Early Day 2,1,1
21,1,26,James Pearce Jr.,Tennessee,2025,9.40,2.131037,Round 2 Tier,Early Day 2,1,1
22,2,44,Donovan Ezeiruaku,Boston College,2025,8.14,2.622417,Round 2 Tier,Early Day 2,1,1
23,2,45,J.T. Tuimoloau,Ohio State,2025,9.34,3.326004,Round 3 Tier,Late Day 2,1,1
24,2,51,Nic Scourton,Texas A&M,2025,NaN,3.341377,Round 3 Tier,Late Day 2,0,1
25,2,52,Oluwafemi Oladejo,UCLA,2025,NaN,4.486793,Round 4 Tier,Early Day 3,0,1
26,2,59,Mike Green,Marshall,2025,NaN,4.169361,Round 4 Tier,Early Day 3,0,1
27,3,66,Ashton Gillotte,Louisville,2025,9.75,1.881108,Round 2 Tier,Early Day 2,1,1


In [26]:
# 2026 evaluation
edges_2026 = prepare_edges_df('edges_drafted_2026.csv', 2026)

X_26_raw = edges_2026[FEATURES_WITH_COLLEGE_ALL].copy()
X_26 = imputer.transform(X_26_raw)
X_26_scaled = scaler.transform(X_26)

pred_26 = np.clip(ridge.predict(X_26_scaled), 1, 8)

# Check if Round column exists and has valid values for evaluation
if 'Round' in edges_2026.columns and edges_2026['Round'].notna().any():
    actual_26 = edges_2026['Round'].astype(int).values
    print('2026 edges:')
    eval_metrics(actual_26, pred_26, '2026')
else:
    print(f'2026 edges (n={len(pred_26)}): Predictions generated (no actual rounds available)')

# Display 2026 predictions
edges_2026_display = edges_2026[['Round', 'Pick', 'Player', 'School', 'Year', 'RAS']].copy()
edges_2026_display['predicted_round'] = pred_26
edges_2026_display['tier_label'] = [pred_round_to_tier(x)[0] for x in pred_26]
edges_2026_display['interpretation'] = [pred_round_to_tier(x)[1] for x in pred_26]
if 'Round' in edges_2026_display.columns and edges_2026_display['Round'].notna().any():
    edges_2026_display['Round'] = edges_2026_display['Round'].astype(int)
# Add contains flags
edges_2026_display['contains_ras'] = edges_2026['contains_ras'].values
edges_2026_display['contains_pff'] = (
    edges_2026['contains_true_pass_set_pass_rush_win_rate'] | 
    edges_2026['contains_pass_rush_win_rate'] | 
    edges_2026['contains_snap_counts_pass_rush'] | 
    edges_2026['contains_stop_percent']
).astype(int)

print('\n2026 drafted edges')
display(edges_2026_display)

2026 edges:
2026 (n=35): MAE=2.0774, RMSE=2.4647, R²=-0.3842, Exact=11.43%, Within-1=37.14%

2026 drafted edges


,Round,Pick,Player,School,Year,RAS,predicted_round,tier_label,interpretation,contains_ras,contains_pff
0,1,1,Rueben Bain Jr.,Miami,2026,NaN,1.000000,Round 1 Tier,True 1st-round grade,0,1
1,1,4,Arvell Reese,Ohio State,2026,NaN,6.786089,Round 7 / UDFA Tier,Fringe draftable,0,1
2,1,9,T.J. Parker,Clemson,2026,NaN,4.368454,Round 4 Tier,Early Day 3,0,1
3,1,10,Keldric Faulk,Auburn,2026,NaN,4.139539,Round 4 Tier,Early Day 3,0,1
4,1,12,David Bailey,Texas Tech,2026,NaN,2.636979,Round 2 Tier,Early Day 2,0,1
5,1,17,Cashius Howell,Texas A&M,2026,NaN,4.734327,Round 4 Tier,Early Day 3,0,1
6,1,26,Romello Height,Texas Tech,2026,NaN,4.036349,Round 4 Tier,Early Day 3,0,1
7,1,30,R Mason Thomas,Oklahoma,2026,NaN,5.756635,Round 6 Tier,Late Day 3,0,0
8,2,37,Joshua Josephs,Tennessee,2026,NaN,5.416654,Round 5 Tier,Mid Day 3,0,1
9,2,49,Matayo Uiagalelei,Oregon,2026,NaN,4.286943,Round 4 Tier,Early Day 3,0,1


In [27]:
# Model results on entire training set (2017–2023), ordered by predicted_round
train_display = df[['Round', 'Pick', 'Player', 'School', 'Year']].copy()
train_display['predicted_round'] = y_pred_train
train_display['tier_label'] = [pred_round_to_tier(x)[0] for x in y_pred_train]
train_display['interpretation'] = [pred_round_to_tier(x)[1] for x in y_pred_train]
train_display = train_display.sort_values('predicted_round').reset_index(drop=True)
train_display

,Round,Pick,Player,School,Year,predicted_round,tier_label,interpretation
0,1.0,1.0,Myles Garrett,Texas A&M,2017,1.000000,Round 1 Tier,True 1st-round grade
1,3.0,73.0,Jordan Willis,Kansas State,2017,1.000000,Round 1 Tier,True 1st-round grade
2,1.0,16.0,Brian Burns,Florida State,2019,1.000349,Round 1 Tier,True 1st-round grade
3,1.0,2.0,Aidan Hutchinson,Michigan,2022,1.176215,Round 1 Tier,True 1st-round grade
4,2.0,32.0,Emmanuel Ogbah,Oklahoma State,2016,1.429108,Round 1 Tier,True 1st-round grade
...,...,...,...,...,...,...,...,...
235,5.0,156.0,Isaiahh Loudermilk,Wisconsin,2021,8.000000,Round 7 / UDFA Tier,Fringe draftable
236,NaN,NaN,Cece Jefferson,Florida,2019,8.000000,Round 7 / UDFA Tier,Fringe draftable
237,NaN,NaN,Bryan Cox Jr.,Florida,2017,8.000000,Round 7 / UDFA Tier,Fringe draftable
238,NaN,NaN,Darius Jackson,Jacksonville State,2018,8.000000,Round 7 / UDFA Tier,Fringe draftable
